<a href="https://colab.research.google.com/github/maxskoda/NR_autoreduce/blob/master/INTER_WebData_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone 'https://github.com/maxskoda/NR_autoreduce/'

Cloning into 'NR_autoreduce'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 63 (delta 13), reused 56 (delta 10), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [ ]:
!pip install requests_html
from bs4 import BeautifulSoup
import requests
import urllib

In [35]:
#@title INTER data viewer
figure_width = 1100 #@param {type:"integer"}
figure_height = 510 #@param {type:"integer"}
RB_Number = "2010748" #@param {type:"string"}

import pandas as pd
import ipywidgets as widgets
import glob
import os
import ntpath
from ipywidgets import Layout
# import chart_studio.plotly as py
from bs4 import BeautifulSoup
import requests
import urllib

import matplotlib.pyplot as plt
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "colab"
import re



url = 'http://inter-reflectometer.is-best.net/RB'+RB_Number+'/'
#'http://maximus.great-site.net/ASCII_Files/'
ext = 'dat'

def get_title(fname):
  url = 'https://shadow.nd.rl.ac.uk/journals/cgi-bin/journal.py?inst=NDXINTER'

  page = requests.get(url, auth=('ktd43279', ''))

  df_list = pd.read_html(page.text) # this parses all the tables in webpages to a list
  df = df_list[0]
  df.head()
  
  run = re.search(r'\d+', fname).group()
  
  return df.loc[df['Run'] == int(run), 'Title'].item()


def listFD(url, ext=''):
    page = requests.get(url).text
    # print(page)
    soup = BeautifulSoup(page, 'html.parser')
    file_list = []
    for node in soup.find_all('a'):
      if node.get('href').endswith(ext):
        if node.get('href').endswith(ext) and (url + '/' + node.get('href')) not in file_list:
          file_list.append(url + '/' + node.get('href'))

    return file_list

# for file in listFD(url, ext):
#     print(file)

def nr_plot(file_list):
    choices = file_list['new']
    output.clear_output(wait=True)
    fig.data = [] # remove all traces
    with output:
        for f in choices:
          X, Y, E = [], [], []
          file = urllib.request.urlopen(url+f)
          for line in file: #NR_autoreduce/ASCII_Files/"+f, 'r'):
            decoded_line = line.decode("utf-8")
            values = [float(s) for s in decoded_line.split()]
            X.append(values[0])
            Y.append(values[1])
            E.append(values[2])

          fig.add_trace(
                      go.Scatter(x=X,
                  y=Y,
                  error_y=dict(
                      type='data', # value of error bar given in data coordinates
                      array=E,
                      visible=True),
                  name=get_title(f))
                  )
        fig.update_xaxes(type="log")
        fig.update_yaxes(type="log")
        fig.show()


# ff = [os.path.basename(x) for x in glob.glob("http://maximus.great-site.net/ASCII_Files/*.txt")]

ff = [ntpath.basename(f) for f in listFD(url, ext)]

selector = widgets.SelectMultiple(
options=ff,
value=[ff[0]],
rows=30,
description='',
disabled=False,
layout=Layout(valign='bottom', width= 'max-content'),
)

fig = go.Figure()

fig.update_layout(
    yaxis = {'exponentformat':'power'},
    margin=dict(l=100, r=20, t=20, b=20),
    modebar= {'orientation': 'v'},
    width=figure_width, height=figure_height)#, 'ticks':'inside'})

output = widgets.Output()

items = [selector, output]
box_layout = Layout(display='flex', flex_flow='row', 
                    # justify_content='space-between', 
                    align_items='stretch',
                    align_content='flex-end')


html = "<h1 style='font-size:20px'> INTER Data Display </h1>"
title = widgets.HTML(value=html)

hb = widgets.HBox(items, layout=box_layout)
# title = widgets.Label(value="INTER Data - RB"+RB_Number)
vb = widgets.VBox([title, hb])
display(vb)
# display(selector)
# display(output)


selector.observe(nr_plot, names='value')
